In [ ]:
!pip install pfhedge

### Last time

In [ ]:
import pfhedge
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
from pfhedge.instruments import BrownianStock
from pfhedge.instruments import EuropeanOption
from pfhedge.nn import BlackScholes

In [ ]:
stock = BrownianStock()
derivative = EuropeanOption(stock, strike=1.0)
m = BlackScholes(derivative)

In [ ]:
derivative.simulate(2)
lm = derivative.log_moneyness()
t = derivative.time_to_maturity()
v = derivative.ul().volatility

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = torch.stack(data, dim=-1)

    def __len__(self):
        return self.data.size(1)

    def __getitem__(self, index):
        return self.data[:, index, :].unsqueeze(1)

ds = MyDataset([lm, t, v])

In [ ]:
outputs = []
for i in ds:
    outputs.append(m(i))

outputs = torch.cat(outputs, dim=-1)
outputs

tensor([[[0.5113, 0.4845, 0.4789, 0.5157, 0.4862, 0.6613, 0.7486, 0.7833,
          0.6604, 0.8916, 0.9397, 0.8242, 0.9030, 0.9627, 0.9469, 0.9853,
          0.9804, 0.9980, 0.9999, 1.0000, 1.0000]],

        [[0.5113, 0.6515, 0.6775, 0.6152, 0.5743, 0.4740, 0.4571, 0.4259,
          0.4055, 0.3408, 0.5193, 0.7640, 0.7278, 0.7305, 0.8530, 0.9165,
          0.8557, 0.7588, 0.5803, 0.9077, 1.0000]]])

In [ ]:
spot = torch.stack([derivative.ul().spot], dim=1)

In [ ]:
pfhedge.nn.functional.pl(spot, outputs, payoff=derivative.payoff())

tensor([-0.0265, -0.0211])

### MLP


In [ ]:
import pfhedge
import torch
from torch.utils.data import Dataset, DataLoader

from pfhedge.instruments import BrownianStock
from pfhedge.instruments import EuropeanOption
from pfhedge.nn import MultiLayerPerceptron

stock = BrownianStock()
derivative = EuropeanOption(stock, strike=1.0)
m = MultiLayerPerceptron(3)

derivative.simulate(2)
lm = derivative.moneyness() # <- Note that this is not log moneyness
t = derivative.time_to_maturity()
v = derivative.ul().volatility

class MyDataset(Dataset):
    def __init__(self, data):
        self.data = torch.stack(data, dim=-1)

    def __len__(self):
        return self.data.size(1)

    def __getitem__(self, index):
        return self.data[:, index, :].unsqueeze(1)

ds = MyDataset([lm, t, v])

In [ ]:
m

MultiLayerPerceptron(
  (0): Linear(in_features=3, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): ReLU()
  (6): Linear(in_features=32, out_features=32, bias=True)
  (7): ReLU()
  (8): Linear(in_features=32, out_features=1, bias=True)
  (9): Identity()
)

### Simple model

In [ ]:
class MLP(torch.nn.Module):

    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
mm = MLP(m)
mm

MLP(
  (model): MultiLayerPerceptron(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=1, bias=True)
    (9): Identity()
  )
)

In [ ]:
mm(torch.randn(2,3))

tensor([[-0.0332],
        [-0.0284]], grad_fn=<AddmmBackward0>)

In [ ]:
for i in ds:
    print(mm(i))

tensor([[[-0.0316]],

        [[-0.0316]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0316]],

        [[-0.0316]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0315]],

        [[-0.0316]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0315]],

        [[-0.0315]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0315]],

        [[-0.0315]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0315]],

        [[-0.0315]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0314]],

        [[-0.0314]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0314]],

        [[-0.0314]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0313]],

        [[-0.0314]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0313]],

        [[-0.0314]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0312]],

        [[-0.0314]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0311]],

        [[-0.0313]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0311]],

        [[-0.0313]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0311]],

        [[-0.0313]]], grad_fn=<ViewBackward0>)
tensor([[[-0.0310]],

        [[-0

In [ ]:
outputs = []
for i in ds:
    outputs.append(mm(i))

outputs = torch.cat(outputs, dim=-1)
outputs

tensor([[[-0.0316, -0.0316, -0.0315, -0.0315, -0.0315, -0.0315, -0.0314,
          -0.0314, -0.0313, -0.0313, -0.0312, -0.0311, -0.0311, -0.0311,
          -0.0310, -0.0310, -0.0310, -0.0310, -0.0310, -0.0309, -0.0309]],

        [[-0.0316, -0.0316, -0.0316, -0.0315, -0.0315, -0.0315, -0.0314,
          -0.0314, -0.0314, -0.0314, -0.0314, -0.0313, -0.0313, -0.0313,
          -0.0312, -0.0312, -0.0312, -0.0311, -0.0311, -0.0311, -0.0310]]],
       grad_fn=<CatBackward0>)

In [ ]:
spot = torch.stack([derivative.ul().spot], dim=1)

In [ ]:
pfhedge.nn.functional.pl(spot, outputs, payoff=derivative.payoff())

tensor([-8.4319e-02,  1.5014e-05], grad_fn=<SubBackward0>)

# Training

In [ ]:
epochs = 2

for i in range(epochs):
    outputs = []
    for j in ds:
        outputs.append(mm(j))

    outputs = torch.cat(outputs, dim=-1)
    print(outputs)

tensor([[[-0.0316, -0.0316, -0.0315, -0.0315, -0.0315, -0.0315, -0.0314,
          -0.0314, -0.0313, -0.0313, -0.0312, -0.0311, -0.0311, -0.0311,
          -0.0310, -0.0310, -0.0310, -0.0310, -0.0310, -0.0309, -0.0309]],

        [[-0.0316, -0.0316, -0.0316, -0.0315, -0.0315, -0.0315, -0.0314,
          -0.0314, -0.0314, -0.0314, -0.0314, -0.0313, -0.0313, -0.0313,
          -0.0312, -0.0312, -0.0312, -0.0311, -0.0311, -0.0311, -0.0310]]],
       grad_fn=<CatBackward0>)
tensor([[[-0.0316, -0.0316, -0.0315, -0.0315, -0.0315, -0.0315, -0.0314,
          -0.0314, -0.0313, -0.0313, -0.0312, -0.0311, -0.0311, -0.0311,
          -0.0310, -0.0310, -0.0310, -0.0310, -0.0310, -0.0309, -0.0309]],

        [[-0.0316, -0.0316, -0.0316, -0.0315, -0.0315, -0.0315, -0.0314,
          -0.0314, -0.0314, -0.0314, -0.0314, -0.0313, -0.0313, -0.0313,
          -0.0312, -0.0312, -0.0312, -0.0311, -0.0311, -0.0311, -0.0310]]],
       grad_fn=<CatBackward0>)


### compute_hedge

In [ ]:
def compute_hedge(model, ds):
    outputs = []
    for i in ds:
        outputs.append(model(i))

    return torch.cat(outputs, dim=-1)

In [ ]:
compute_hedge(mm, ds)

tensor([[[-0.0316, -0.0316, -0.0315, -0.0315, -0.0315, -0.0315, -0.0314,
          -0.0314, -0.0313, -0.0313, -0.0312, -0.0311, -0.0311, -0.0311,
          -0.0310, -0.0310, -0.0310, -0.0310, -0.0310, -0.0309, -0.0309]],

        [[-0.0316, -0.0316, -0.0316, -0.0315, -0.0315, -0.0315, -0.0314,
          -0.0314, -0.0314, -0.0314, -0.0314, -0.0313, -0.0313, -0.0313,
          -0.0312, -0.0312, -0.0312, -0.0311, -0.0311, -0.0311, -0.0310]]],
       grad_fn=<CatBackward0>)

### compute_portfolio

In [ ]:
def compute_portfolio(model, ds, derivative):

    spot = torch.stack([derivative.ul().spot], dim=1)
    unit = compute_hedge(model, ds)

    return pfhedge.nn.functional.pl(spot, unit)

In [ ]:
compute_portfolio(mm, ds, derivative)

tensor([-2.5796e-03,  1.5014e-05], grad_fn=<SumBackward1>)

In [ ]:
def compute_portfolio_2(model, ds, derivative):

    spot = torch.stack([derivative.ul().spot], dim=1)
    unit = compute_hedge(model, ds)

    return pfhedge.nn.functional.pl(spot, unit, payoff=derivative.payoff())

In [ ]:
compute_portfolio_2(mm, ds, derivative)

tensor([-8.4319e-02,  1.5014e-05], grad_fn=<SubBackward0>)

### fit()

In [ ]:
def fit(model, ds, derivative):

    optimizer = torch.optim.Adam(model.parameters())

    for i in range(100):
        optimizer.zero_grad()
        cash = compute_portfolio_2(model, ds, derivative)
        loss = torch.mean(cash)
        loss.backward()
        optimizer.step()

        if i % 1 == 0:
            print(loss)



In [ ]:
fit(mm, ds, derivative)

tensor(0.0062, grad_fn=<MeanBackward0>)
tensor(0.0056, grad_fn=<MeanBackward0>)
tensor(0.0049, grad_fn=<MeanBackward0>)
tensor(0.0043, grad_fn=<MeanBackward0>)
tensor(0.0036, grad_fn=<MeanBackward0>)
tensor(0.0030, grad_fn=<MeanBackward0>)
tensor(0.0024, grad_fn=<MeanBackward0>)
tensor(0.0018, grad_fn=<MeanBackward0>)
tensor(0.0012, grad_fn=<MeanBackward0>)
tensor(0.0007, grad_fn=<MeanBackward0>)
tensor(0.0001, grad_fn=<MeanBackward0>)
tensor(-0.0004, grad_fn=<MeanBackward0>)
tensor(-0.0010, grad_fn=<MeanBackward0>)
tensor(-0.0015, grad_fn=<MeanBackward0>)
tensor(-0.0020, grad_fn=<MeanBackward0>)
tensor(-0.0025, grad_fn=<MeanBackward0>)
tensor(-0.0029, grad_fn=<MeanBackward0>)
tensor(-0.0034, grad_fn=<MeanBackward0>)
tensor(-0.0039, grad_fn=<MeanBackward0>)
tensor(-0.0043, grad_fn=<MeanBackward0>)
tensor(-0.0048, grad_fn=<MeanBackward0>)
tensor(-0.0052, grad_fn=<MeanBackward0>)
tensor(-0.0056, grad_fn=<MeanBackward0>)
tensor(-0.0060, grad_fn=<MeanBackward0>)
tensor(-0.0064, grad_fn=<Me

### bigger dataset

In [ ]:
derivative.simulate(2000)
lm = derivative.moneyness()
t = derivative.time_to_maturity()
v = derivative.ul().volatility

ds = MyDataset([lm, t, v])

In [ ]:
compute_hedge(mm, ds)